# ESiWACE3 Compression Lab in WASM

**Note:** This WASM-based version of the compression lab has only been tested in new Chrome and Firefox browsers.

## Fetch the dataset

In [ ]:
import pyodide

response = await pyodide.http.pyfetch("/files/hplp_ml_q_dx=2.0.grib")
with open("hplp_ml_q_dx=2.0.grib", "wb") as file:
    file.write(await response.bytes())

## Load the dataset

In [ ]:
import fcpy

In [ ]:
ds = fcpy.open_dataset("hplp_ml_q_dx=2.0.grib")
ds = ds[["q"]]
ds

## Visualise the dataset

In [ ]:
fcpy.suite.plot_spatial_single(
    ds["q"].sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Baseline",
)

## Run a simple compressor

In [ ]:
compressed_linquant_2 = fcpy.suite.run_compressor_single(ds["q"], fcpy.compressors.LinQuantization(), bits=2)

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_2.sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compressed (bits=2)",
)

## Plot the simple compression error

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_2.sel(dict(hybrid=1)) - ds["q"].sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compression Error (bits=2)",
)

## Run a better compressor

In [ ]:
compressed_linquant_24 = fcpy.suite.run_compressor_single(ds["q"], fcpy.compressors.LinQuantization(), bits=24)

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_24.sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compressed (bits=24)",
)

## Plot the better compression error

In [ ]:
fcpy.suite.plot_spatial_single(
    compressed_linquant_24.sel(dict(hybrid=1)) - ds["q"].sel(dict(hybrid=1)),
    ds,
    "q",
    metric="Compression Error (bits=24)",
)